# Students Performance Analysis by Region
This notebook analyzes student performance by region.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load CSV file
df = pd.read_csv(r"C:/Users/DELL/Downloads/ethiopian_students_dataset.csv")

# View first 5 rows
print(df.head())

# Access a column
print(df.columns)